# Statistical Models: Time Series Forecasting

In [1]:
path = "https://frenzy86.s3.eu-west-2.amazonaws.com/timeseries/Data/airline_passengers.csv"

In [2]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

#path = 'airline-passengers.csv'

df_passengers = pd.read_csv(path,
                            parse_dates=["Month"],
                            index_col="Month",
                            )

df_passengers = df_passengers.asfreq('MS')
df_passengers.columns = ['Passengers']
df_passengers

,Passengers
Month,
1949-01-01,112
1949-02-01,118
1949-03-01,132
1949-04-01,129
1949-05-01,121
...,...
1960-08-01,606
1960-09-01,508
1960-10-01,461


## Create DataFrame for storing forecasts

In [3]:
horizon = 48 # The next 4 years (12 months x 4 years)
date_end_raw = df_passengers.index[-1]

date_range = pd.date_range(start=date_end_raw, periods=horizon + 1, freq='MS')
date_range = date_range[1:]
df_forecast_base = pd.DataFrame(index=date_range)
df_forecast_base['Passengers'] = np.nan
df_forecast_base

,Passengers
1961-01-01,NaN
1961-02-01,NaN
1961-03-01,NaN
1961-04-01,NaN
1961-05-01,NaN
1961-06-01,NaN
1961-07-01,NaN
1961-08-01,NaN
1961-09-01,NaN
1961-10-01,NaN


## Prepare Historical Data

In [4]:
data = df_passengers['Passengers'].values
data

array([112, 118, 132, 129, 121, 135, 148, 148, 136, 119, 104, 118, 115,
       126, 141, 135, 125, 149, 170, 170, 158, 133, 114, 140, 145, 150,
       178, 163, 172, 178, 199, 199, 184, 162, 146, 166, 171, 180, 193,
       181, 183, 218, 230, 242, 209, 191, 172, 194, 196, 196, 236, 235,
       229, 243, 264, 272, 237, 211, 180, 201, 204, 188, 235, 227, 234,
       264, 302, 293, 259, 229, 203, 229, 242, 233, 267, 269, 270, 315,
       364, 347, 312, 274, 237, 278, 284, 277, 317, 313, 318, 374, 413,
       405, 355, 306, 271, 306, 315, 301, 356, 348, 355, 422, 465, 467,
       404, 347, 305, 336, 340, 318, 362, 348, 363, 435, 491, 505, 404,
       359, 310, 337, 360, 342, 406, 396, 420, 472, 548, 559, 463, 407,
       362, 405, 417, 391, 419, 461, 472, 535, 622, 606, 508, 461, 390,
       432])

## Models of ARIMA Components

### Autoregression (AR)


In [7]:
from statsmodels.tsa.ar_model import AutoReg
pd.options.plotting.backend = "plotly"

def ar_forecast(data, lags, horizon):
    model = AutoReg(data, lags=lags)
    model_fit = model.fit()
    forecast = model_fit.predict(start=len(data), end=len(data)+horizon-1)
    return forecast

df_forecast_arima_components = df_forecast_base.copy()
df_forecast_arima_components['Autoregression (AR)'] = ar_forecast(data, 12, horizon)

pd.concat([df_passengers, df_forecast_arima_components]).plot()

### Moving Average (MA)

In [8]:
from statsmodels.tsa.arima.model import ARIMA

def ma_forecast(data, order, horizon):
    model = ARIMA(data, order=(0, 0, order))
    model_fit = model.fit()
    forecast = model_fit.predict(start=len(data), end=len(data)+horizon-1)
    return forecast

df_forecast_arima_components['Moving Average (MA)'] = ma_forecast(data, horizon, horizon)

fig = pd.concat([df_passengers, df_forecast_arima_components]).plot(title='[AR]I[MA] Components Forecast')
fig = fig.update_layout(legend_title_text='Model Type')
fig

/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



## ARIMA & SARIMA

### ARIMA

In [9]:
def arima_forecast(data, order, horizon):
    model = ARIMA(data, order=order)
    model_fit = model.fit()
    forecast = model_fit.predict(start=len(data), end=len(data)+horizon-1)
    return forecast

df_forecast_arima = df_forecast_base.copy()
df_forecast_arima['ARIMA'] = arima_forecast(data, order=(2,1,2), horizon=horizon)

pd.concat([df_passengers, df_forecast_arima]).plot()

/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



### Seasonal ARIMA (SARIMA)

In [10]:
def sarima_forecast(data, order, seasonal_order, horizon):
    model = ARIMA(data, order=order, seasonal_order=seasonal_order)
    model_fit = model.fit()
    forecast = model_fit.predict(start=len(data), end=len(data)+horizon-1)
    return forecast

df_forecast_arima['SARIMA'] = sarima_forecast(data, (0, 1, 2), (1, 1, 2, 12), horizon)

fig = pd.concat([df_passengers, df_forecast_arima]).plot(title='ARIMA Models Forecast')
fig = fig.update_layout(legend_title_text='Model Type')
fig

## Exponential Smoothing

In [11]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing

### Simple Exponential Smoothing

In [14]:
def simple_exp_smoothing_forecast(data, horizon):
    model = SimpleExpSmoothing(data)
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=horizon)
    return forecast

df_forecast_exponential = df_forecast_base.copy()
df_forecast_exponential['Simple Exponential Smoothing'] = simple_exp_smoothing_forecast(data, horizon)

pd.concat([df_passengers, df_forecast_exponential]).plot()

### Holt's Exponential Smoothing

In [15]:
def holt_forecast(data, horizon):
    model = ExponentialSmoothing(data, trend='add')
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=horizon)
    return forecast

df_forecast_exponential['Holt'] = holt_forecast(data, horizon)

pd.concat([df_passengers, df_forecast_exponential]).plot()

### Holt-Winters Exponential Smoothing


In [16]:
def holt_winters_forecast(data, horizon, seasonal_periods=12):
    model = ExponentialSmoothing(data, trend='add', seasonal='add', seasonal_periods=seasonal_periods)
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=horizon)
    return forecast


df_forecast_exponential['Holt-Winters'] = holt_winters_forecast(data, horizon, seasonal_periods=12)

fig = pd.concat([df_passengers, df_forecast_exponential]).plot(title='Exponential Models Forecast')
fig = fig.update_layout(legend_title_text='Model Type')
fig

## All together

In [17]:
list_df = [df_forecast_arima_components, df_forecast_arima, df_forecast_exponential]
list_df = [df.drop(columns='Passengers') for df in list_df]

df_forecast = pd.concat(list_df, axis=1)
df_forecast = pd.concat([df_passengers, df_forecast], axis=1)

df_forecast.plot()